# 빌트인 도구

- 랭체인 제공
- 도구와 툴킷
- 툴 -> 단일 도구 / 툴킷 -> 여러 도구가 묶여 있다
- 파일관리 툴킷 : 파일삭제 툴, 파일읽기 툴 ...

## 1. 검색 툴 사용해보기

```
uv add langchain_tavily
```

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

project_name = "wanted_2nd_agent_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [3]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_tavily import TavilySearch

In [ ]:
# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 2. 도구 생성 (웹서치 툴)
tavily_search = TavilySearch()

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 툴만 사용해서 사실을 확인하고 한국어로 간단하게 답한다"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm = llm,
    tools = [tavily_search],
    prompt = prompt
)

# 5. executor 설정
executor = AgentExecutor(
    agent = agent,
    tools = [tavily_search],
    verbose = True
)

In [8]:
# 실행하기
result = executor.invoke({"input" : "서울의 오늘 주요 뉴스 3개 요약해줘"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '서울 오늘 주요 뉴스', 'time_range': 'day', 'topic': 'news'}`


{'query': '서울 오늘 주요 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.cbsnews.com/newyork/video/first-alert-weather-clearing-skies-for-2nd-half-of-week/', 'title': 'First Alert Weather: Clearing skies for 2nd half of week - CBS News', 'score': 0.15870851, 'published_date': 'Wed, 15 Oct 2025 10:23:00 GMT', 'content': "livelive: CBS News Minnesota### CBS News New York livelive: CBS News New York### CBS News Philadelphia 04:426h ago### President Trump threatens to withhold federal funds if Mamdani wins NYC mayoral election 01:598h ago### First Alert Weather: Cooler temps on the way in New York - 10/14/25 02:339h ago### CC Sabathia named grand marshal of 2025 TCS NYC Marathon 02:2512h ago### New York Public Library exhibit highlights Puerto Ricans in comic books 01:0012h ago 03:1012h ago### Mayor Adams cons

In [9]:
result


{'input': '서울의 오늘 주요 뉴스 3개 요약해줘',
 'output': "서울의 오늘 주요 뉴스 3개는 다음과 같습니다.\n\n1. 김민석 국무총리가 반복되는 반중 집회를 '자해 행위'라고 비판하며 관련 부처에 법과 원칙에 따라 엄정 대응할 것을 지시했습니다. 경찰은 APEC 행사장 주변에 경찰력을 집중하고 외국인 비하 집회에 엄정 대응할 계획입니다.\n\n2. 서울의 날씨는 맑아지고 있으며, 이번 주 후반에는 기온이 내려갈 것으로 예상됩니다.\n\n3. 아시아 증시는 미국 금리 인하 기대감에 힘입어 상승세를 보이고 있으며, 미중 무역 긴장 완화 기대도 영향을 미치고 있습니다."}

## 2. 파일 저장 툴 사용해보기

In [ ]:
# 1. 파일 저장 툴 가져오기
from langchain_community.agent_toolkits import FileManagementToolkit

root_dir = "../agent_result"
file_tools = FileManagementToolkit(root_dir = root_dir).get_tools()
file_tools

[CopyFileTool(root_dir='../agent_result'),
 DeleteFileTool(root_dir='../agent_result'),
 FileSearchTool(root_dir='../agent_result'),
 MoveFileTool(root_dir='../agent_result'),
 ReadFileTool(root_dir='../agent_result'),
 WriteFileTool(root_dir='../agent_result'),
 ListDirectoryTool(root_dir='../agent_result')]

In [15]:
# 2. 웹서치 툴 가져오기
tavily_search = TavilySearch(max_results=5)

In [ ]:
# 3. 툴 합치기
total_tool = file_tools + [tavily_search]
total_tool

[CopyFileTool(root_dir='../agent_result'),
 DeleteFileTool(root_dir='../agent_result'),
 FileSearchTool(root_dir='../agent_result'),
 MoveFileTool(root_dir='../agent_result'),
 ReadFileTool(root_dir='../agent_result'),
 WriteFileTool(root_dir='../agent_result'),
 ListDirectoryTool(root_dir='../agent_result'),
 TavilySearch(max_results=5, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))]

In [29]:
# 4. 시스템 프롬프트 설정하기
SYSTEM_PROMPT = """
    너는 '파일시스템 툴'과 '웹검색 툴'만을 사용하는 어시스턴트야.

    [규칙]
    1. 최신 뉴스/링크는 Tavily로 검색한다
    2. 사용자가 csv 파일로 저장을 요청하면, 다음 필드 헤더로 csv를 만든다
        date, query, title, url, source, snippet
    3. 파일명은 {YYYYMMDD}.csv로 저장한다
    4. 같은 파일명이 있다면, 덮어쓰는 것은 하지 말고, 기존 내용을 유지한 채 병합하는 형식으로 해라
    5. 저장 후 내용을 확인해서 가장 아래 5줄 정도 정보를 출력한다
    
    모든 답변은 한국어로 간결하게 해라
"""

In [19]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%Y%m%d")

In [21]:
get_today()

'20251016'

In [30]:
# 앞과 동일, tool과 prompt만 변경해주자

# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 2. 도구 생성
# total_tool

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm = llm,
    tools = total_tool,
    prompt = prompt
)

# 5. executor 설정
executor = AgentExecutor(
    agent = agent,
    tools = total_tool,
    verbose = True
)

In [31]:
result = executor.invoke({
    "input" : "오늘 AI 관련 뉴스 최신 5개만 요약해주고 그 내용을 csv로 저장해줘",
    "YYYYMMDD" : get_today()
})

print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'AI', 'time_range': 'day', 'topic': 'general', 'search_depth': 'basic'}`


{'query': 'AI', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.wired.com/story/the-ai-industrys-scaling-obsession-is-headed-for-a-cliff/', 'title': "The AI Industry's Scaling Obsession Is Headed for a Cliff | WIRED", 'content': 'A new study from MIT suggests the biggest and most computationally intensive AI models may soon offer diminishing returns compared to smaller models. Leaps in efficiency, like those seen with DeepSeek’s remarkably low-cost model in January, have already served as a reality check for the AI industry, which is accustomed to burning massive amounts of compute. While the MIT team’s prediction might not hold if, for example, new training methods like reinforcement learning produce surprising new results, they suggest that big AI firms will have less of an edge in the

## 3. 뉴스 검색후 sqlite 저장하는 agent

In [32]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../agent_result/new.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm).get_tools()
toolkit

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000127CF2B86D0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000127CF2B86D0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000127CF2B86D0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [33]:
SYSTEM_PROMPT = """
    당신은 외부 정보를 검색하고, 그 결과를 SQLite 데이터베이스에 저장하는 AI 비서입니다.

    당신은 두 가지 도구를 사용할 수 있습니다:

    1. **웹 검색 도구 (TavilySearch)**: 인터넷에서 외부 정보를 검색합니다. 데이터베이스에 없는 정보나 최신 정보가 필요할 때 사용하세요.

    2. **SQL 도구 (SQLDatabaseToolkit)**: SQLite 데이터베이스에 접근할 수 있습니다. 정보를 조회하거나 저장(INSERT)할 수 있습니다.

    ### 당신의 작업 흐름은 다음과 같습니다:

    - 사용자의 질문에 대해 데이터베이스에 해당 정보가 있는지 먼저 확인합니다.
    - 정보가 없거나 부족할 경우, **TavilySearch 도구를 사용해 웹 검색**을 통해 필요한 데이터를 수집합니다.
    - 데이터베이스에 테이블이 없을경우 적절히 만듭니다.
    - 검색한 정보를 **적절히 정리하고**, 필요한 경우 텍스트를 요약하거나 구조화해서 **SQLite 데이터베이스에 INSERT**합니다.
    - 이후, 저장된 데이터를 기반으로 사용자에게 답변합니다.

    **항상 구조화된 데이터를 저장하는 것을 우선시**하고, 중복 저장을 피하세요.

    가능하다면 다음 형식을 따르세요:
    - 웹에서 얻은 정보 → 정리 → SQL INSERT 문 생성 → 데이터베이스에 저장

    모든 작업은 투명하고, 사용자가 이해할 수 있도록 설명과 함께 수행하세요.

"""

In [34]:
# 앞과 동일, tool과 prompt만 변경해주자

# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 2. 도구 생성
total_tool = toolkit + [tavily_search]

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm = llm,
    tools = total_tool,
    prompt = prompt
)

# 5. executor 설정
executor = AgentExecutor(
    agent = agent,
    tools = total_tool,
    verbose = True
)

In [35]:
result = executor.invoke({"input" : "최신 NPU 관련 뉴스 5개 찾아서 알려줘"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`



Invoking: `tavily_search` with `{'query': 'latest NPU news', 'search_depth': 'advanced', 'topic': 'general', 'include_favicon': True}`


{'query': 'latest NPU news', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.techpowerup.com/news-tags/NPU', 'title': "News Posts matching 'NPU'", 'content': '# Intel "Nova Lake-S" Core Ultra 3, Ultra 5, Ultra 7, and Ultra 9 Core Configurations Surface\n\n# QNAP Unveils the New QAI-M100 and QAI-U100 Edge AI Accelerators\n\n# AMD and Stability AI Enable Local AI Image Generation on NPU-Powered Laptops\n\n# Intel Skips NPU Upgrade for Arrow Lake Refresh, AMD Cancels Medusa Halo in Latest Rumors\n\n# Intel Plans "Arrow Lake Refresh" for H2\u202f2025 with Higher Clocks and Upgraded NPU\n\n# AMD UDNA Graphics Architecture to Power Next-Gen Xbox and PlayStation [...] # News Posts matching #NPU\n\n# Synaptics Launches the Next Gene